In [ ]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
# import tensorflow as tf
import pandas as pd



In [16]:
# Pfad zur .jsonl-Datei
file_path = r'C:\Users\carme\OneDrive - Appelt Steuerberatung\Carmen Appelt\Master\Semester 3\DASP\data\train_disapere.jsonl'  # Ersetzen Sie durch den tatsächlichen Pfad

# JSON-Lines-Datei in ein DataFrame laden
df = pd.read_json(file_path, lines=True)
df = df[['text', 'aspect']]
df = df.rename(columns={'aspect': 'labels'})
df = df.rename(columns={'text': 'data'})
print(df)

                                                   data  \
0     The present paper proposes a fast approximatio...   
1     This is typically a bottleneck in deep learnin...   
2     The approximation is a sparse two-layer mixtur...   
3     The paper lacks rigor and the writing is of lo...   
4                            See a list of typos below.   
...                                                 ...   
5211  It is difficult to judge the performance of th...   
5212  3.\tThe proposed method, which decomposes a pr...   
5213  For example, representing a movie or news arti...   
5214  In this way, the proposed method can be tested...   
5215  4.\tFor the final purpose, comparing problem s...   

                         labels  
0                          none  
1                          none  
2                          none  
3                   asp_clarity  
4                          none  
...                         ...  
5211              asp_substance  
5212               

In [17]:
df["labels"].unique()

# Häufigkeit der Labels zählen
label_counts = df['labels'].value_counts()

print("Verteilung der Labels:")
print(label_counts)


Verteilung der Labels:
labels
none                         2554
asp_substance                 825
asp_clarity                   566
asp_soundness-correctness     536
asp_originality               200
asp_motivation-impact         174
asp_meaningful-comparison     173
asp_replicability             120
arg_other                      68
Name: count, dtype: int64


In [18]:
#Encode label for easy identification.
df["encoded_cat"] = df["labels"].astype("category").cat.codes

data_texts = df["data"].to_list() # Features (not tokenized yet)
data_labels = df["encoded_cat"].to_list() # Labels

In [25]:
df

,data,labels,encoded_cat
0,The present paper proposes a fast approximatio...,none,8
1,This is typically a bottleneck in deep learnin...,none,8
2,The approximation is a sparse two-layer mixtur...,none,8
3,The paper lacks rigor and the writing is of lo...,asp_clarity,1
4,See a list of typos below.,none,8
...,...,...,...
5211,It is difficult to judge the performance of th...,asp_substance,7
5212,"3.\tThe proposed method, which decomposes a pr...",none,8
5213,"For example, representing a movie or news arti...",none,8
5214,"In this way, the proposed method can be tested...",none,8


In [19]:
# Data Split (train 79%, val 20%, test 1%)

from sklearn.model_selection import train_test_split
 
# Split Train and Validation data
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=0, shuffle=True)
 
# Keep some data for inference (testing)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=0.01, random_state=0, shuffle=True)

In [22]:
print(len(train_texts)/len(data_texts))
print(len(val_texts)/len(data_texts))
print(len(test_texts)/len(data_texts))

print(train_texts)

0.7917944785276073
0.20015337423312884
0.008052147239263804
['However, the experimental comparison is not fair, the description of the model (e.g. how Choquet is integrated into the model and help to learn “intermediate meaningful results”) is not clear, some claims are not true.', '- A sentence could be added to explain the truncation trick in the abstract directly since it is simple to understand and is key to the quality of the results.', '- The overall method seems to be not very principled, and requires a lot of "tweaks and tunes", with additional losses and regularizers, to work.', 'The main technical results of the paper are Theorems 5-8 which compute the statistics of the covariance of the activations and the gradients.', 'Hence, these overly simplified and unrealistic assumptions make the task too trivial.', "- Currently there is no experimental evaluation of the claims, which would be valuable given that the setting doesn't immediately apply in the normal batch normalization 

In [ ]:
#Daten tokenisieren
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
import tensorflow as tf

#Erstellen der TensorFlow-Datasets

# Konvertieren Sie die Labels in Tensoren
train_labels = tf.convert_to_tensor(train_labels)
val_labels = tf.convert_to_tensor(val_labels)

# Erstellen Sie TensorFlow-Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))



In [ ]:
#  Initialisieren des Modells

num_labels = 9  

model = TFDistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=num_labels
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
#Kompilieren des Modells

from tensorflow.keras.metrics import Precision, Recall

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(
    optimizer=optimizer,
    loss=model.hf_compute_loss,  # Interne Verlustfunktion des Modells
    metrics=['accuracy', Precision(), Recall()]
)

NameError: name 'Precision' is not defined